# 0. 개발 환경 및 라이브러리 버전

In [1]:
import platform
import sys

print(f"Operating System: {platform.system()} {platform.release()}")
print(f"Python Version: {sys.version}")

Operating System: Darwin 22.5.0
Python Version: 3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]


In [8]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [21]:
# !pip freeze

numpy==1.26.4  
pandas==2.2.3  
scikit-learn==1.6.1  
xgboost==2.1.3  
lightgbm==4.5.0  
catboost==1.2.7  

# 1. 함수 정의

**main 함수에 사용되는 함수를 정의하는 코드**입니다.  **main 함수는 아래 '2. Main 함수 실행'에서 확인**하실 수 있습니다.  
  
기본적으로 Seed 하나당 XGBoost 모델의 5-fold cross-validation 한 결과를 확인할 수 있도록 작성했고,  
최종적으로 선택한 submission 파일은 Seed 5개로 시드 앙상블하여 제출했습니다.  
  
모델 앙상블 실험을 위해 XGBoost, LightGBM, CatBoost를 선택할 수 있도록 작성되어 있으며,  
Seed Number는 팀원들이 임의로 선택, 하이퍼 파라미터는 optuna를 여러 번 활용해 채택한 결과입니다.

## 1-0. Config

In [9]:
import sys
import os
import random
from datetime import datetime
import time

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import (
    OrdinalEncoder,
    TargetEncoder
)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

## 1-1. set_seed

In [10]:
def set_seed(seed):
    random.seed(seed)                      # Python 기본 random 모듈의 시드 고정
    np.random.seed(seed)                   # NumPy 난수 생성기 시드 고정

    os.environ['PYTHONHASHSEED'] = str(seed)  # Python 해시 함수 시드 고정 (Python >= 3.3)

    try:
        import torch
        torch.manual_seed(seed)            # PyTorch CPU 시드 고정
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)   # PyTorch GPU 시드 고정
            torch.cuda.manual_seed_all(seed)  # PyTorch 모든 GPU 시드 고정
        torch.backends.cudnn.deterministic = True  # CUDNN을 결정적으로 설정
        torch.backends.cudnn.benchmark = False     # 성능에 영향을 줄 수 있음
    except ImportError:
        pass  # PyTorch가 설치되어 있지 않은 경우 생략

    try:
        import tensorflow as tf
        tf.random.set_seed(seed)           # TensorFlow 시드 고정
    except ImportError:
        pass  # TensorFlow가 설치되어 있지 않은 경우 생략

여러 라이브러리의 Seed를 고정하는 함수입니다.

## 1-2. read_data

In [11]:
def read_data(data_path):
    df_train = pd.read_csv(data_path + 'train.csv')
    df_test = pd.read_csv(data_path + 'test.csv')
    df_submit = pd.read_csv(data_path + 'sample_submission.csv')

    return df_train, df_test, df_submit

train, test, sample_submission 데이터 파일을 읽어옵니다.

## 1-3. feature_engineering

In [12]:
def feature_engineering(df_input):
    df = df_input.copy()

    '''
    통합 전처리
    '''

    # '시술 당시 나이'변수 수치형으로 변환
    df['시술 당시 나이'] = df['시술 당시 나이'].map({
        '만18-34세':1, '만35-37세':2, '만38-39세':3, '만40-42세':4,
        '만43-44세':5, '만45-50세':6, '알 수 없음':7
    })

    # '~횟수'변수 수치형으로 변환
    count_features = [feat for feat in df.columns if '횟수' in feat]
    for feat in count_features:
        df[feat] = df[feat].apply(lambda val : int(val[0])) # n회/n회 이상 (str) -> n (int)

    # '정자,난자 기증자 나이'변수 수치형으로 변환
    for feat in ['난자 기증자 나이', '정자 기증자 나이']:
        df[feat] = df[feat].apply(lambda val : int(val[1:3]) if val[0] == '만' else -1)

    # 배아 생성 주요 이유 다중 선택 변환 (5개 더미 컬럼 생성)
    categories = ['현재 시술용', '배아 저장용', '기증용', '난자 저장용']
    for category in categories:
        df[f'배아 이유_{category}'] = df['배아 생성 주요 이유'].apply(lambda x: 1 if pd.notna(x) and category in x else 0)

    # '출산 성공률', '임신 성공률' 파생변수 생성
    df['출산 성공률'] = df['총 출산 횟수'] / df['총 임신 횟수']
    df['임신 성공률'] = df['총 임신 횟수'] / df['총 시술 횟수']
    
    '''
    IVF / DI 개별 전처리
    '''

    ivf_df = df[df['시술 유형'] == 'IVF'].copy()
    di_df = df[df['시술 유형'] == 'DI'].copy()

    # DI인 경우 남성 주 ~ 부부 부 불임 원인 flip
    for feat in ['남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불임 원인 - 정자 농도']:
        di_df[feat] = di_df[feat].apply(lambda x: 1-x)

    # 확정 결측치 처리
    for feat in ['PGS 시술 여부', 'PGD 시술 여부', '착상 전 유전 검사 사용 여부']:
        ivf_df[feat] = ivf_df[feat].fillna(0)

    df[df['시술 유형'] == 'IVF'] = ivf_df
    df[df['시술 유형'] == 'DI'] = di_df

    return df

1. **'시술 당시 나이' 변수 수치형 변환 :** 오름차순으로 정수 값 치환
2. **'횟수' 변수 수치형 변환 (총 임신 횟수 등) :** 정수값만 추출
3. **'정자, 난자 기증자 나이' 변수 수치형으로 변환 :** 정수값만 추출
4. **'배아 생성 주요 이유' 변수 dummy variable 변환 :** '연구용'의 경우 관측치 개수가 매우 적어서 따로 생성하지 않음
5. **'출산 성공률', '임신 성공률' 파생 변수 생성 :** 행 내에서 '총 출산 횟수', '총 임신 횟수', '총 시술 횟수' 활용 계산
6. **'불임 원인' 변수들 중 IVF와 DI에서 target 분포가 다른 경우 flip :** 0 -> 1, 1 -> 0
7. **'PGS 시술 여부', 'PGD 시술 여부', '착상 전 유전 검사 사용 여부' 결측 처리 :** IVF에만 해당하는 변수들이므로, DI는 '해당없음', IVF는 '시술이나 사용하지 않음'의 의미로 다르게 구분하기 위해 DI는 -1, IVF는 0으로 처리

## 1-4. make_feature_lists

In [ ]:
def make_feature_lists(df):
    base_features = []
    cat_features = []
    num_features = []

    removal_features = {'신선 배아 사용 여부', '동결 배아 사용 여부'} # '신선 배아 사용 여부', '동결 배아 사용 여부' drop
    removal_features |= set(df.columns[df.nunique(dropna=False) == 1])  # 모든 row가 다 결측 or 한가지 값인 변수 제거

    for col in df.columns:
        if col == 'ID' or col == '임신 성공 여부':
            continue

        if col in removal_features:
            continue

        base_features.append(col)
        if df[col].dtype == 'object':
            cat_features.append(col)
        else:
            num_features.append(col)

    return base_features, cat_features, num_features, removal_features

변수 선택, encoding, fillna, CatBoost 모델 인자 등으로 사용하기 위한 변수 리스트 생성 함수입니다.

## 1-5. encoding_cat_feautres

In [ ]:
def encoding_cat_features(df_train, df_test, cat_features, config_params, model_name, seed):
    target = '임신 성공 여부'
    train_encoded, test_encoded = df_train.copy(), df_test.copy()

    # catboost는 모델 내 자체 encoder 활용 
    if model_name == 'CB':
        return train_encoded, test_encoded

    if model_name == 'XGB':
        model_encoder = config_params['xgb_encoder']
    elif model_name == 'LGBM':
        model_encoder = config_params['lgb_encoder']
    else:
        model_encoder = None

    match model_encoder:
        case 'ordinal':
            encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
            train_encoded[cat_features] = encoder.fit_transform(df_train[cat_features])
        case 'target':
            encoder = TargetEncoder(random_state=seed)
            train_encoded[cat_features] = encoder.fit_transform(df_train[cat_features], df_train[target])
        case _:
            print("Error: Invalid Encoder Name")
            raise

    test_encoded[cat_features] = encoder.transform(df_test[cat_features])

    return train_encoded, test_encoded

모델별 encoder 선택 (ordinal , target) 함수입니다. 

## 1-6. filling_missing_values

In [ ]:
def filling_missing_values(df_input, cat_features, num_features, model_name):
    df = df_input.copy()
    
    # catboost는 범주형 변수 UNK 처리, 나머지 모델들은 인코딩한 뒤 -1로 처리 
    if model_name == 'CB':
        for cat_feature in cat_features:
            df[cat_feature] = df[cat_feature].fillna('UNK')
            df[cat_feature] = df[cat_feature].astype(str)

    for num_feature in num_features:
        df[num_feature] = df[num_feature].fillna(-1)

    return df

결측치는 유효하지 않은 값 (범주형 변수: UNK, 수치형 변수: -1)으로 처리하여 실제 관측치와 구분했습니다.

## 1-7. model_kfold 

In [16]:
def model_kfold(df_input, base_features, cat_features, config_params, model_params, model_name, seed):
    df = df_input.copy()
    target = '임신 성공 여부'

    skf = StratifiedKFold(n_splits=config_params['k_fold'], shuffle=True, random_state=seed)
    models = []
    scores = []

    for k_fold, (train_idx, valid_idx) in enumerate(skf.split(df[base_features], df[target])):
        X_train, y_train = df[base_features].iloc[train_idx], df[target].iloc[train_idx].astype(int)
        X_valid, y_valid = df[base_features].iloc[valid_idx], df[target].iloc[valid_idx].astype(int)

        match model_name:
            case 'CB':
                model = CatBoostClassifier(**model_params)
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_valid, y_valid)],
                    cat_features=cat_features,
                )
            case 'XGB':
                model = XGBClassifier(**model_params)
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_valid, y_valid)],
                    verbose=False
                )
            case 'LGBM':
                model = LGBMClassifier(**model_params)
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_valid, y_valid)],
                )
            case _:
                print("Invalid Model Name")
                raise

        models.append(model)

        # print performances
        print('[Valid] ', end=' ')
        y_pred = model.predict_proba(X_valid)[:,1]
        score = roc_auc_score(y_valid, y_pred)
        print(f'Fold #{k_fold + 1} Score: {score:.4f}')

        scores.append(score)

    print(f'Avg. Score of validset: {np.mean(scores)}')
    print(f'Std. Score of validset: {np.std(scores)}')
    print()

    return models

k-fold cross-validation으로 모델을 학습한 뒤, valid score의 평균과 표준편차를 출력합니다.  
Stratified K-fold를 선택하여 fold별로 target 분포를 동일하게 추출했습니다.

## 1-8. predict_test

In [17]:
def predict_test(models, df_input, base_features):
    df = df_input.copy()

    X_test = df[base_features]
    y_probs = np.zeros((X_test.shape[0], 2))

    for i, model in enumerate(models):
        y_probs += model.predict_proba(X_test) / len(models)

    return y_probs

test 데이터를 fold별 모델으로 추론한 뒤, 그 평균으로 임신 성공 확률을 계산합니다.

## 1-9. make_submission

In [18]:
def make_submission(y_probs, df_submit):
    result_path = 'Results'

    if not os.path.exists(result_path):
        os.makedirs(result_path)

    # 현재 날짜와 시간을 얻습니다.
    now = datetime.now()

    month = now.month
    day = now.day
    hour = now.hour
    minute = now.minute

    submission_time = f"{month:02d}{day:02d}_{hour:02d}{minute:02d}"

    y_prob = np.zeros((df_submit.shape[0], 2))

    for y_prob_model in y_probs:
        y_prob += y_prob_model / len(y_probs)

    # 제출 파일 작성
    df_submit['probability'] = y_prob[:, 1]

    # 제출 파일 저장
    df_submit.to_csv(f"./{result_path}/submission_{submission_time}.csv", index=False)

시드별 임신 성공확률의 평균으로 최종 probability를 계산합니다.  
submission 파일의 버전 관리를 위해 실행 시간을 파일명에 함께 기재합니다.

# 2. main 함수 실행 

## 2-1. main

In [19]:
def main(config_params, model_param_dict):
    start_time = time.time()  # 시작 시간 기록

    y_probs = []

    for seed in config_params['seed_list']:
        print('\n'+'='*30)
        print(f"Seed Number : {seed}")
        print("="*30)
        for model_name, model_params in model_param_dict.items():
            print(f"<Model : {model_name}>")

            # seed 고정
            set_seed(seed)

            # 데이터 셋 읽기
            df_train, df_test, df_submit = read_data(config_params['data_path'])

            # 데이터 전처리
            df_train = feature_engineering(df_train)
            df_test = feature_engineering(df_test)

            base_features, cat_features, num_features, removal_features = make_feature_lists(df_train)

            # 범주형 변수 인코딩
            df_train, df_test = encoding_cat_features(df_train, df_test, cat_features, config_params, model_name, seed)

            # 결측 처리
            df_train = filling_missing_values(df_train, cat_features, num_features, model_name)
            df_test = filling_missing_values(df_test, cat_features, num_features, model_name)

            # train 데이터 학습
            models = model_kfold(df_train, base_features, cat_features, config_params, model_params, model_name, seed)

            # 제출 파일 생성
            y_prob = predict_test(models, df_test, base_features)
            y_probs.append(y_prob)

    make_submission(y_probs, df_submit)

    end_time = time.time()  # 종료 시간 기록
    elapsed_time = end_time - start_time  # 총 실행 시간 계산

    minutes, seconds = divmod(elapsed_time, 60)  # 분, 초로 변환
    print(f"총 실행 시간: {int(minutes)}분 {seconds:.2f}초")  # 실행 시간 출력

seed_list, model_list를 순환하며 모델+시드 앙상블한 뒤 submission 파일을 생성합니다.  
최종 제출 파일에는 XGBoost 모델 하나에 대한 5시드 앙상블을 진행했습니다.

## 2-2. main 함수 실행 

In [20]:
if __name__ == "__main__":

    config_params = {
        'seed_list': [777, 2, 14, 63, 2000],
        'data_path': "./Data/",
        'k_fold': 5,
        'models': ['XGB'],
        # CatBoost : CB
        # XGBoost : XGB
        # LightGBM : LGBM
        'xgb_encoder': 'ordinal', # ordinal / target
        'lgb_encoder': 'target'
    }

    model_param_setting = {
        'XGB': {
            'n_estimators': 2357,
            'learning_rate': 0.016535403312726182,
            'max_depth': 7,
            'subsample': 0.6887677799128866,
            'colsample_bytree': 0.6879076928246035,
            'reg_lambda': 0.28211678046980465,
            'reg_alpha': 6.879354177483545,
            'gamma': 2.413362122265103,
            'objective': 'binary:logistic',
            'scale_pos_weight': 1,
            'verbosity': 0,
        },
        'LGBM': {
            'n_estimators': 2290,
            'learning_rate': 0.007317488794263521,
            'max_depth': 8,
            'subsample': 0.6704092605499218,
            'colsample_bytree': 0.3607645380846825,
            'reg_lambda': 0.5897801081491055,
            'reg_alpha': 6.882099222744256,
            'objective': 'binary',
            'metric': 'auc',
            'class_weight': 'balanced',
            #'early_stopping_rounds': 50,
            'verbose': -1,
        },
        'CB': {
            'iterations': 1000,
            'learning_rate': 0.03,
            'depth': 6,
            'objective': 'Logloss',
            'auto_class_weights': 'Balanced',
            'verbose': 0,
        },
    }

    model_param_dict = dict()
    for model_name in config_params['models']:
        model_param_dict[model_name] = model_param_setting[model_name]

    main(config_params, model_param_dict)


Seed Number : 777
<Model : XGB>
[Valid]  Fold #1 Score: 0.7384
[Valid]  Fold #2 Score: 0.7420
[Valid]  Fold #3 Score: 0.7376
[Valid]  Fold #4 Score: 0.7416
[Valid]  Fold #5 Score: 0.7428
Avg. Score of validset: 0.7404880968025566
Std. Score of validset: 0.002063187912857618


Seed Number : 2
<Model : XGB>
[Valid]  Fold #1 Score: 0.7420
[Valid]  Fold #2 Score: 0.7388
[Valid]  Fold #3 Score: 0.7392
[Valid]  Fold #4 Score: 0.7423
[Valid]  Fold #5 Score: 0.7380
Avg. Score of validset: 0.7400497815215938
Std. Score of validset: 0.00176950084224428


Seed Number : 14
<Model : XGB>
[Valid]  Fold #1 Score: 0.7408
[Valid]  Fold #2 Score: 0.7394
[Valid]  Fold #3 Score: 0.7401
[Valid]  Fold #4 Score: 0.7402
[Valid]  Fold #5 Score: 0.7406
Avg. Score of validset: 0.7402477093606528
Std. Score of validset: 0.0004824819346555312


Seed Number : 63
<Model : XGB>
[Valid]  Fold #1 Score: 0.7394
[Valid]  Fold #2 Score: 0.7384
[Valid]  Fold #3 Score: 0.7413
[Valid]  Fold #4 Score: 0.7407
[Valid]  Fold #5

seed_list = \[777, 2, 14, 63, 2000\]를 선택해 총 5개 시드에 대해 시드 앙상블 진행.  
models = \['XGB'\]를 선택해 XGBoost 모델만 사용.  